#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load huggingface embedding model (``model_name="sentence-transformers/all-mpnet-base-v2"``)
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [34]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

# ADD HERE YOUR CODE
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\sschn\OneDrive\Documents\Studium\3.Semester\Technische Informatik\Chatbot\TeamNovaBot\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
text = "This is a test document."

# ADD HERE YOUR CODE
# Perform vector search
query_vector = embedding_model.embed_query(text)

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 768
[-0.048951826989650726, -0.039862070232629776, -0.021562794223427773, 0.009908574633300304, -0.03810390084981918, 0.012684382498264313, 0.043494462966918945, 0.07183389365673065, 0.00974861066788435, -0.006987075321376324]


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: ``PyPDFLoader``
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [51]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
# ADD HERE YOUR CODE
loader = PyPDFLoader(file_path=pdf_doc)

# Load and split documents in chunks
# ADD HERE YOUR CODE
pages_chunked = RecursiveCharacterTextSplitter(chunk_size =10000,chunk_overlap =20).split_documents(documents=loader.load())

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(text):
    # Remove surrogate pairs
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text
def clean_and_create_document(chunk):
    cleaned_text= clean_text(chunk.page_content)
    return Document(page_content=cleaned_text,metadata=chunk.metadata)

pages_chunked_cleaned = [clean_and_create_document (chunk) for chunk in pages_chunked]

print(pages_chunked[0])

page_content='Aurélien GéronHands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent SystemsSECOND EDITION
Boston Farnham Sebastopol Tokyo Beijing Boston Farnham Sebastopol Tokyo Beijing' metadata={'source': './AI_Book.pdf', 'page': 2}


In [52]:
print(pages_chunked[1])

page_content='978-1-492-03264-9
[LSI]Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurélien Géron
Copyright © 2019 Aurélien Géron. All rights reserved.
Printed in the United States of America.
Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles ( http://oreilly.com ). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com .
Editor:  Nicole Tache
Interior Designer:  David FutatoCover Designer:  Karen Montgomery
Illustrator:  Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649  for release details.
The O’Reilly logo 

In [53]:
print(f"Number of text chunks: {len(pages_chunked)}")

Number of text chunks: 501


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)

In [74]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# ADD HERE YOUR CODE
# Create a collection
collection = client.get_or_create_collection("collection_name")

# ADD HERE YOUR CODE
# Create chromadb
vector_db_from_client = Chroma(client=client,
                               collection_name="collection_name",
                               embedding_function=embedding_model,
                               )

In [75]:
from uuid import uuid4

#uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned[:50]))]

# ADD HERE YOUR CODE
#vector_db_from_client.add_texts(pages_chunked_cleaned,id=uuids)

uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned[:50]))]

# ADD HERE YOUR CODE
vector_db_from_client.add_documents(documents=pages_chunked_cleaned[:50],id=uuids)

['60d63bf1-6ec1-4dfd-8b6e-8c897c1e288c',
 'f882fd53-ca70-4bd5-a34f-7393ea912f67',
 '4dcaa9c4-7333-4fbd-8aeb-f5c7b408a59d',
 '991b9185-8bd4-4a4f-9b2a-d3a5c5955526',
 'b36e7d6d-324c-49f5-86c2-b4f9b8bced12',
 'a1d03718-8239-4316-93be-77f0df43de51',
 '0a0194b4-c100-4eab-ab2a-972ce1fb65e9',
 '0bd2a86d-7607-433d-ac33-177a112f16bf',
 'a8821427-3a48-404a-bd44-32a4e4241a84',
 'c7ee5398-6338-4d0f-871d-9f096a657675',
 '58fba749-7637-4654-8906-551592db45c3',
 'b169df47-3bab-4b9f-b326-57580234328b',
 'c1ae8698-2bfd-4142-b20e-6d7150941045',
 '58aba155-fa34-496a-9adc-96dba642564b',
 '568ada68-b02f-4a21-a95a-06b053808cb6',
 '25258cc1-19bc-4d69-867e-dea27c406422',
 '9a5dfc49-bc7f-440f-a3a2-d007cb57294f',
 '202b3ad8-2333-40cd-8f3c-f5f8c12bd94f',
 'd6e691ea-8869-40ad-a101-dbc0479484b7',
 '1e5fde1d-99e8-4793-9889-8ddfd4595e63',
 '81b426bb-470d-4055-99a1-847bb9e80425',
 '01ab3f90-d654-47ad-b3ea-2b88ecc667f9',
 '6dcc3d81-9ff4-4c2c-8e23-246e7a821064',
 'ea0d292f-1bf1-4671-a44d-2d6ec4f4f9ec',
 '532a030b-3236-

In [76]:
client.count_collections()

1

In [89]:
# client.delete_collection("ai_model_book")

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents


**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)

In [79]:
search_query="Types of Machine Learning Systems"

results = vector_db_from_client.similarity_search(query=search_query,k=3)

for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")

Exception: {"error":"InvalidArgumentError","message":"Expected where to have exactly one operator, got {}"}